# Defensive Player of the Year Award

In [180]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

awards_data = pd.read_csv("../../initial_data/awards_players.csv")
teams_data = pd.read_csv("../../initial_data/teams.csv")
players_teams_clean = pd.read_csv("../awards_data/players_teams_clean.csv")
players_info = pd.read_csv("../../initial_data/players_clean.csv")

## Create Defensive Player of the Year Data

In [181]:
# Join dataset players_teams_clean data with teams_data (team_wins, team_losses, rank, playoff_made)
# Note: players_teams_clean has 'team' column, teams_data has 'tmID' column
defpoty_data = players_teams_clean.merge(
    teams_data[['year', 'tmID', 'won', 'lost', 'rank', 'playoff', 'confID']], 
    left_on=['year', 'team'], 
    right_on=['year', 'tmID'], 
    how='left'
)

# Rename columns for clarity
defpoty_data.rename(columns={
    'won': 'team_wins', 
    'lost': 'team_losses', 
    'rank': 'conference_rank', 
    'playoff': 'playoff_made'
}, inplace=True)

# Drop the duplicate tmID column (we already have 'team')
defpoty_data.drop(columns=['tmID'], inplace=True)

# Change Playoff from N / Y to 0 / 1
defpoty_data['playoff_made'] = defpoty_data['playoff_made'].map({'Y': 1, 'N': 0})
# Change ConfID to numeric  EA / WE to 0 / 1
defpoty_data['confID'] = defpoty_data['confID'].astype('category').cat.codes

# Add player physical attributes
defpoty_data = defpoty_data.merge(
    players_info[['bioID', 'pos', 'height', 'weight']],
    left_on='playerID',
    right_on='bioID',
    how='left'
)
defpoty_data.drop(columns=['bioID'], inplace=True)

# Add Defensive POTY winner field (1 if player won that year, 0 otherwise)
defpoty_winners = awards_data[awards_data['award'] == 'Defensive Player of the Year'][['year', 'playerID']]
defpoty_data['DefPOTY_winner'] = defpoty_data.apply(
    lambda row: 1 if ((defpoty_winners['year'] == row['year']) & 
                      (defpoty_winners['playerID'] == row['playerID'])).any() else 0,
    axis=1
)

print(f"Defensive POTY data shape: {defpoty_data.shape}")
print(f"\nColumns: {defpoty_data.columns.tolist()}")
print(f"\nDefensive POTY winners in dataset: {defpoty_data['DefPOTY_winner'].sum()}")
print(f"\nSample data:")
print(defpoty_data.head())
print(f"\nDefensive POTY winners:")
print(defpoty_data[defpoty_data['DefPOTY_winner'] == 1][['playerID', 'year', 'team', 'pos', 'height', 'weight']])

# Save csv (relative path from current location)
defpoty_data.to_csv("defpoty_data.csv", index=False)
print(f"\n✅ Saved to defpoty_data.csv")


Defensive POTY data shape: (1876, 27)

Columns: ['playerID', 'year', 'team', 'minutes', 'games_played', 'total_points', 'total_rebounds', 'total_assists', 'points_per_min', 'assists_per_min', 'rebounds_per_min', 'steals_per_min', 'blocks_per_min', 'turnovers_per_min', 'FG%', 'FT%', 'Three%', 'Three Rate', 'team_wins', 'team_losses', 'conference_rank', 'playoff_made', 'confID', 'pos', 'height', 'weight', 'DefPOTY_winner']

Defensive POTY winners in dataset: 10

Sample data:
     playerID  year team  minutes  games_played  total_points  total_rebounds  \
0  abrossv01w     2  MIN      846            26           343             174   
1  abrossv01w     3  MIN      805            27           314             146   
2  abrossv01w     4  MIN      792            30           318             141   
3  abrossv01w     5  MIN      462            22           146              74   
4  abrossv01w     6  MIN      777            31           304             107   

   total_assists  points_per_min  a

## Define features to use in the models

In [182]:
# Define feature columns for the model
# Using year N-1 stats to predict year N
feature_columns = [
    'minutes','games_played','total_rebounds','rebounds_per_min','steals_per_min',
    'blocks_per_min','turnovers_per_min','team_wins','team_losses','conference_rank',
    'playoff_made','confID', 'pos', 'height', 'weight'
]

print("Features for Defensive Player of the Year prediction:")
for i, col in enumerate(feature_columns, 1):
    print(f"{i}. {col}")


Features for Defensive Player of the Year prediction:
1. minutes
2. games_played
3. total_rebounds
4. rebounds_per_min
5. steals_per_min
6. blocks_per_min
7. turnovers_per_min
8. team_wins
9. team_losses
10. conference_rank
11. playoff_made
12. confID
13. pos
14. height
15. weight


## Prediction Models

### 1. Logistic Regression

In [183]:
# Load data and prepare for training
# Strategy: Use year N-1 stats to predict year N defpoty
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

defpoty_data = pd.read_csv("defpoty_data.csv")

# Create training dataset: year N-1 stats → year N defpoty
# For each player's stats in year N-1, check if they won defpoty in year N
defpoty_data['next_year'] = defpoty_data['year'] + 1

# Get defpoty winners for next year
defpoty_next_year = defpoty_data[defpoty_data['DefPOTY_winner'] == 1][['year', 'playerID']].copy()
defpoty_next_year.rename(columns={'year': 'next_year'}, inplace=True)

# Merge to create target variable
defpoty_data = defpoty_data.merge(
    defpoty_next_year,
    on=['next_year', 'playerID'],
    how='left',
    indicator=True
)
defpoty_data['defpoty_winner_next_year'] = (defpoty_data['_merge'] == 'both').astype(int)
defpoty_data.drop(columns=['_merge'], inplace=True)

# Filter to defpoty candidates only
defpoty_candidates = defpoty_data[
    (defpoty_data['minutes'] >= 500) &
    (defpoty_data['games_played'] >= 20)
].copy()

print(f"Total candidates: {len(defpoty_candidates)}")
print(f"defpoty winners (next year): {defpoty_candidates['defpoty_winner_next_year'].sum()}")
print(f"Years: {sorted(defpoty_candidates['year'].unique())}")


Total candidates: 866
defpoty winners (next year): 7
Years: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


#### Training the model

In [184]:
# Train Logistic Regression Model

# EXAMPLE: To predict year 7 -> exclude year 6 from training (add it to testing), use all other years
train_years = [1, 2, 3, 4, 5, 6, 7, 8]
test_years = [9]

train_data = defpoty_candidates[defpoty_candidates['year'].isin(train_years)].copy()
test_data = defpoty_candidates[defpoty_candidates['year'].isin(test_years)].copy()

X_train = train_data[feature_columns].copy()
y_train = train_data['defpoty_winner_next_year']
X_test = test_data[feature_columns].copy()
y_test = test_data['defpoty_winner_next_year']

# Encode position as numeric category
X_train['pos'] = X_train['pos'].astype('category').cat.codes
X_test['pos'] = X_test['pos'].astype('category').cat.codes

print(f"Training: {len(X_train)} candidates, {y_train.sum()} defpoty winners")
print(f"Testing: {len(X_test)} candidates, {y_test.sum()} defpoty winners")

# Scale and train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

test_data['defpoty_probability'] = y_pred_proba
test_data['defpoty_prediction'] = y_pred

print(f"\n✅ Model trained!")
print(f"Train accuracy: {model.score(X_train_scaled, y_train):.3f}")
print(f"Test accuracy: {model.score(X_test_scaled, y_test):.3f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Not DefPOTY', 'DefPOTY']))


Training: 690 candidates, 6 defpoty winners
Testing: 89 candidates, 1 defpoty winners

✅ Model trained!
Train accuracy: 0.935
Test accuracy: 0.888

Classification Report:
              precision    recall  f1-score   support

 Not DefPOTY       0.99      0.90      0.94        88
     DefPOTY       0.00      0.00      0.00         1

    accuracy                           0.89        89
   macro avg       0.49      0.45      0.47        89
weighted avg       0.98      0.89      0.93        89



In [185]:
# Show test results
test_results = test_data.sort_values('defpoty_probability', ascending=False)
print("\nYear 9 Stats → Year 10 defpoty Predictions:")
print(f"{'Rank':<6}{'Player':<25}{'Probability':<12}{'Actual defpoty'}")
print("-" * 60)
for idx, (_, row) in enumerate(test_results.head(10).iterrows(), 1):
    is_defpoty = "✅" if row['defpoty_winner_next_year'] == 1 else ""
    print(f"{idx:<6}{row['playerID']:<25}{row['defpoty_probability']:.4f}{'':6}{is_defpoty}")



Year 9 Stats → Year 10 defpoty Predictions:
Rank  Player                   Probability Actual defpoty
------------------------------------------------------------
1     hornbal01w               0.9899      
2     anosini01w               0.9871      
3     beviltu01w               0.9224      
4     swoopsh01w               0.9187      
5     parkeca01w               0.8860      
6     smithta01w               0.8474      
7     leslili01w               0.8011      
8     fordch01w                0.5718      
9     jacksla01w               0.5142      
10    whaleli01w               0.4535      


In [186]:
# Diagnostic: Check feature importance and actual defpoty stats
print("\n🔍 Feature Importance (Top 10):")
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': model.coef_[0]
}).sort_values('coefficient', ascending=False)
print(feature_importance.head(10))

# Check if actual DefPOTY is in the test data
# Get the predicted year from test_years variable
predicted_year = test_years[0] + 1  # test_years contains stats year, add 1 for predicted year
stats_year = test_years[0]

print(f"\n🔍 Actual Year {predicted_year} DefPOTY in test data:")
actual_defpoty = test_data[test_data['defpoty_winner_next_year'] == 1]
if len(actual_defpoty) > 0:
    print(f"DefPOTY: {actual_defpoty.iloc[0]['playerID']}")
    print(f"Probability: {actual_defpoty.iloc[0]['defpoty_probability']:.4f}")
    print(f"Year {stats_year} stats:")
    print(actual_defpoty[['total_rebounds', 'steals_per_min', 'blocks_per_min', 'games_played', 'team_wins', 'conference_rank']].iloc[0])
else:
    print("❌ DefPOTY not found in test candidates!")
    print(f"Checking if DefPOTY exists in year {predicted_year} data at all...")
    # Check who won DefPOTY in the predicted year
    predicted_year_defpoty = defpoty_data[(defpoty_data['year'] == predicted_year) & (defpoty_data['DefPOTY_winner'] == 1)]
    if len(predicted_year_defpoty) > 0:
        print(f"Year {predicted_year} DefPOTY: {predicted_year_defpoty.iloc[0]['playerID']}")
        # Check if they have stats in the stats_year
        stats_year_data = defpoty_data[(defpoty_data['year'] == stats_year) & (defpoty_data['playerID'] == predicted_year_defpoty.iloc[0]['playerID'])]
        if len(stats_year_data) > 0:
            print(f"\nTheir Year {stats_year} stats:")
            print(stats_year_data[['minutes', 'games_played', 'total_rebounds', 'steals_per_min', 'blocks_per_min', 'team_wins']].iloc[0])
            print(f"\nFiltered out because:")
            if stats_year_data.iloc[0]['minutes'] < 500:
                print(f"  ❌ Minutes: {stats_year_data.iloc[0]['minutes']} < 500")
            if stats_year_data.iloc[0]['games_played'] < 20:
                print(f"  ❌ Games: {stats_year_data.iloc[0]['games_played']} < 20")
        else:
            print(f"❌ DefPOTY has no Year {stats_year} stats!")



🔍 Feature Importance (Top 10):
             feature  coefficient
5     blocks_per_min     1.996475
4     steals_per_min     1.934698
3   rebounds_per_min     1.539651
7          team_wins     1.398738
9    conference_rank     1.182171
0            minutes     1.046520
2     total_rebounds     0.762611
10      playoff_made     0.454207
8        team_losses     0.344459
12               pos     0.281341

🔍 Actual Year 10 DefPOTY in test data:
DefPOTY: catchta01w
Probability: 0.3629
Year 9 stats:
total_rebounds     157.000000
steals_per_min       0.070605
blocks_per_min       0.015850
games_played        25.000000
team_wins           17.000000
conference_rank      4.000000
Name: 284, dtype: float64


#### Predict Defensive Player of the Year for New Year

In [187]:
# Predict defpoty for new year using previous year stats
# Example: year_10_players_teams_test.csv → use year 9 stats → predict year 10 defpoty

new_year_file = "../../yearly_data/year_10_players_teams_test.csv"
predict_year = 10
stats_year = predict_year - 1

print(f"Predicting Year {predict_year} defpoty using Year {stats_year} stats")

# Load new year player list
new_year_players = pd.read_csv(new_year_file)
print(f"Players in year {predict_year}: {len(new_year_players)}")

# Get year N-1 stats for these players
player_stats = defpoty_data[
    (defpoty_data['year'] == stats_year) &
    (defpoty_data['playerID'].isin(new_year_players['playerID']))
].copy()

# Filter to defpoty candidates
candidates = player_stats[
    (player_stats['minutes'] >= 500) &
    (player_stats['games_played'] >= 20)
].copy()

print(f"defpoty candidates: {len(candidates)}")

# Predict
X_new = candidates[feature_columns].copy()
# Encode position as numeric category
X_new['pos'] = X_new['pos'].astype('category').cat.codes
X_new_scaled = scaler.transform(X_new)
candidates['defpoty_probability'] = model.predict_proba(X_new_scaled)[:, 1]
candidates = candidates.sort_values('defpoty_probability', ascending=False)

# Show top predictions
print(f"\nYear {predict_year} defpoty Predictions (based on Year {stats_year} stats):")
print(f"{'Rank':<6}{'Player':<25}{'Team':<6}{'Probability':<12}{'PPG':<8}{'Wins'}")
print("-" * 70)
for idx, (_, row) in enumerate(candidates.head(15).iterrows(), 1):
    ppg = row['total_points'] / row['games_played']
    print(f"{idx:<6}{row['playerID']:<25}{row['team']:<6}{row['defpoty_probability']:.4f}{'':6}{ppg:.1f}{'':6}{int(row['team_wins'])}")

print(f"\n🏆 Predicted defpoty: {candidates.iloc[0]['playerID']}")

# Get actual defpoty and their probability
actual_defpoty_id = defpoty_data[(defpoty_data['year'] == predict_year) & (defpoty_data['DefPOTY_winner'] == 1)]['playerID'].values[0]
actual_defpoty_prob = candidates[candidates['playerID'] == actual_defpoty_id]['defpoty_probability']
if len(actual_defpoty_prob) > 0:
    print(f"✅ Actual defpoty: {actual_defpoty_id} // Calculated Probability: {actual_defpoty_prob.values[0]:.4f}")
else:
    print(f"✅ Actual defpoty: {actual_defpoty_id} // Not in candidates (filtered out)")


Predicting Year 10 defpoty using Year 9 stats
Players in year 10: 165
defpoty candidates: 87

Year 10 defpoty Predictions (based on Year 9 stats):
Rank  Player                   Team  Probability PPG     Wins
----------------------------------------------------------------------
1     hornbal01w               DET   0.9899      5.4      22
2     anosini01w               MIN   0.9871      9.2      16
3     beviltu01w               IND   0.9224      5.8      17
4     parkeca01w               LAS   0.8860      18.5      20
5     smithta01w               PHO   0.8474      11.1      16
6     leslili01w               LAS   0.8011      15.1      20
7     fordch01w                DET   0.5718      10.1      22
8     jacksla01w               SEA   0.5142      20.2      22
9     whaleli01w               CON   0.4535      14.0      21
10    catchta01w               IND   0.3629      13.3      17
11    perkiji01w               CHI   0.3399      17.0      12
12    brunsre01w               SAC   0.28